# Анализ данных сервиса аренды самокатов GoFast.

В вашем распоряжении данные сервиса аренды самокатов GoFast. Нам предоставленны данные о пользователях сервиса аренды самокатов из нескольких городов, а также информация об их поездках. Кроме того, у нас есть файл рассказывающий о двух видах использования сервиса: с подпиской и без. Подписка стоит 199 рублей в месяц, за это пользователь получает бесплатный старт и 6 рублей за минуту поездки. Без подписки пользователю придется заплатить за старт поездки 50 рублей и 8 рубдей за каждую потраченную на поездку минуту. 

Цель нашего проекта проанализировать данные о пользователях сервиса аренды самокатов и о их поездках для того чтобы выявить есть ли влияние подписки на длительность поездки. Более того, в нашем проекте будут проверены гипотезы, которые помогут улучшить сервис аренды самокатов. (Пользователи с подпиской больше времени на поездки; Расстояние, которое проезжают пользователи с подпиской за одну поездку, не превышает 
3130 метров; Помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки.)

Наша работа будет состоять из 7 шагов:
1. Загрузка данных
2. Предобработка данных
3. Исследовательский анализ данных
4. Объединение данных
5. Подсчёт выручки
6. Проверка гипотез
7. Вывод

В итоге работы мы получим информацию о том, влияет ли подписка на длину поездки или нет. Также мы подтвердим или опровергнем наши гипотезы. 

In [1]:
#Импортируем все необходимые библиотеки и откроем файл.
import warnings
warnings.filterwarnings('ignore')
from scipy import stats as st
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#### Шаг 1. Загрузка данных

In [2]:
#Просмотрим все датасеты, которые у нас есть.
try:
    users = pd.read_csv('users_go.csv', sep=',')
    rides = pd.read_csv('rides_go.csv', sep=',')
    subscriptions = pd.read_csv('subscriptions_go.csv', sep=',')
except:
    users = pd.read_csv('/datasets/users_go.csv', sep=',')
    rides = pd.read_csv('/datasets/rides_go.csv', sep=',')
    subscriptions = pd.read_csv('/datasets/subscriptions_go.csv', sep=',')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/users_go.csv'

In [ ]:
users.head(10)

In [ ]:
users.info()

В файле users мы можем посмотреть количество пользователей, которые берут в прокат самокаты. В нём также находится информация об уникальном ID пользователя, его имени, возраста, городе и типе подписке. 

В таблице 1565 строк с данными о пользователях приложения. Столбцы названы корректно, тип данных также соответствующий. 

In [ ]:
rides.head(10)

In [ ]:
rides.info()

В файле rides мы можем посмотреть количество пользователей, которые берут в прокат самокаты. По этому файлу мы можем посмотреть какое количество времени и сколько километров длилась каждая поездка пользователя, а также дату поездки. Для того чтобы понять к какому пользователю относится так или иная поездка, там нужно обратить внимение на ID пользователя. 

В таблице 18068 строк с данными о каждой из поездок пользователей. Столбцы названы корректно. Данные в столбце distance и duration позже приведем к целочисленным для коректных расчетов. Также данные в столбце date преобразуем к формату даты pandas.


In [ ]:
subscriptions.head(10)

In [ ]:
subscriptions.info()

В файле subscriptions мы можем посмотреть как подписка влияет на стоимость аренды самоката, в этом файле указано два типа подписки, стоимость одной минуты поездки по каждой подписке, стоимость начала поездки и стоимость ежемесячного платежа для каждой из подписок. 

В таблице 2 строки с данными о видах подписки. Столбцы названы корректно, тип данных также соответствующий. 

#### Шаг 2. Предобработка данных

In [ ]:
#Приведем столбец date к типу даты pandas
rides['date'] = pd.to_datetime(rides['date'], format='%Y-%m-%d')

In [ ]:
#Добавим столбец в котором указан месяц публикации объявления.
rides['month'] = rides['date'].dt.month

In [ ]:
rides['date'].describe()

Судя по данной информации, нам предоставили данные за полный 2021 год. 

In [ ]:
#Проверим дайл users на наличие пропусков 
users.isna().sum()

In [ ]:
#Проверим дайл rides на наличие пропусков 
rides.isna().sum()

In [ ]:
#Проверим дайл subscriptions на наличие пропусков 
subscriptions.isna().sum()

In [ ]:
#Проверим дайл users на наличие дубликатов 
users.duplicated().sum()

In [ ]:
#Проверим дайл rides на наличие дубликатов 
rides.duplicated().sum()

In [ ]:
#Проверим дайл subscriptions на наличие дубликатов 
subscriptions.duplicated().sum()

Проверка данных на дубликаты показала, что в файле users 31 дубликат. Рассмотрим их более детально. 

In [ ]:
users.value_counts().head(31)

In [ ]:
#Удалим дубликаты методом drop_duplicates и сделаем проверку. 
users = users.drop_duplicates()
users.duplicated().sum()

В результате проделанной предобработки данных, мы удалили 31 дубликат в файле users, привели столбец date в файле rides к типу даты pandas, также узнали за какой период нам предоставленны данные (за полный 2021 год).

#### Шаг 3. Исследовательский анализ данных

In [ ]:
#Посмотрим какие города наиболее часто встречаются среди пользователей. 
users['city'].value_counts()

In [ ]:
#Построим график, который поможет нам правильно визуализировать данные. 
users['city'].hist(bins=15, figsize=(12,5)) 
plt.title("Города, встречающиеся наиболее часто среди пользователей")
plt.xlabel("Города")
plt.ylabel("Количество пользователей")
plt.show()

Наиболее популярные города для аренды самоката - Пятигорск, Екатеринбург и Ростов-на-Дону. Возможно в этих городах у сервиса аренды самокатов GoFast больше всего самокатов, поэтому и поездок там больше. 

In [ ]:
#Проанализируем соотношение пользователей с подпиской и без подписки.
have_subscription = (users['subscription_type'] == 'ultra').sum()/users['subscription_type'].count()
not_have_subscription = (users['subscription_type'] == 'free').sum()/users['subscription_type'].count()
print('Доля людей с подпиской:', have_subscription)
print('Доля людей без подписки:', not_have_subscription)

In [ ]:
vals = [have_subscription, not_have_subscription]
labels = ["Доля людей с подпиской", "Доля людей без подписки"]
fig, ax = plt.subplots()
ax.pie(vals, labels=labels, autopct='%1.1f%%')
plt.title("Соотношение пользователей с подпиской и без подписки")
plt.show()

Пользователей, которые использует приложение без подписки больше на 9%, чем тех, кто использует с пропиской. 

In [ ]:
#Изучим возраст пользователей.
users['age'].value_counts()

In [ ]:
#Построим график, который поможет нам правильно визуализировать данные. 
users['age'].hist(bins=30, figsize=(12,5))
plt.title("Возраст, встречающиеся наиболее часто среди пользователей")
plt.xlabel("Возраст")
plt.ylabel("Количество пользователей")
plt.show()

Наиболее распространённый возраст пользователей приложения составляет 25 лет. Большенство пользователей находяться в возрасте от 21 до 29 лет. Самому младшему пользователю 12 лет, а старшему 43 года. 

In [ ]:
# Проанализируем расстояние, которое пользователь преодолел за одну поездку
rides['distance'].hist(bins=100, figsize=(12,5), range=(0,7000)) 
plt.title("Длина поездки, которая чаще всего встречалась среди пользователей")
plt.xlabel("Расстояние, которое пользователь проехал за поездку")
plt.ylabel("Частота поездок")
plt.show()

На графике видно два пиковых значения: на растоянии в 700 метров и 3000 метров. Возможно это связанно с оплатой поездки. В некоторых сервисах аренды самоката пользователь платит за старт поездки и за первые 5 минут поездки. Возможно для того чтобы не платить поминутные поездки люди берут самокат на 5 минут и как раз этого хватает чтобы проехать небольшое растояние до 1 км. тогда это объясняет пиковое значение в 700 метров. Второй пик можно объяснить тем, что люди чаще всего берут самокат на среднее растояние, а на более длинную поездку предпочитают иные виды транспорта (такси, автобус или метро). 

In [ ]:
rides['distance'].describe()

В среднем пользователи проезжают три километра за одну поездку. Минимальное растояние поездки составляет менее метра, возможно это связанно с тем, что люди нечеянно начинаю поездку и сразу ее заканчивают. Максимальное растояние поездки составляет 7,2 км. 

In [ ]:
#Посмотрим продолжительность поездок.
rides['duration'].hist(bins=100, figsize=(12,5), range=(0,45)) 
plt.title("Время поездки, которое чаще всего встречалось среди пользователей")
plt.xlabel("Время за которое пользователь проехал за поездку")
plt.ylabel("Частота поездок")
plt.show()

На графике есть необычный пик около 0.5 минут. Скорее всего это аномалия и нам следует расмотреть такие поездки более детально.

In [ ]:
rides[rides['duration'] < 1 ]

In [ ]:
rides[rides['duration'] < 1].describe()

Расстояние которое люди приодолели за пол минуты согласно данным равно около 5100 метров. Скорее всего это техническая ошибка. Нам следует удалить данные аномалии для более граммотного анализа и чистой статистики. Кроме того следует передать данную информацию коллегам, которые занимаются техническими ошибками, для того чтобы устанить данные аномалии в дальнейшей статистики. 

In [ ]:
rides = rides[rides.duration > 1] 

In [ ]:
rides['duration'].hist(bins=100, figsize=(12,5), range=(0,45)) 
plt.title("Время поездки, которое чаще всего встречалось среди пользователей")
plt.xlabel("Время за которое пользователь проехал за поездку")
plt.ylabel("Частота поездок")
plt.show()

In [ ]:
rides['duration'].describe()

Среднее время поездки составляет 17,8 минут. Самая короткая поездка длилась пол минуты. Самая долгая поездка была 40 минутная. 

После удаления аномалий изменилось время самой короткой поездки, она длилась 2 минуты. Самая долгая поездка была 40 минутная. Среднее время поездки составляет 17,8 минут.

Усредненный портрет пользователя и его поездки:
1. Скорее всего пользователь из Пятигорска, Екатеренбурга или Ростова-на-Дону
2. Есть больше вероятности, что пользователь без подписки. 
3. Средний возраст пользователя 25 лет. 
4. Растояние и время средней поездки составляет 3 км и 17,8 минут. 

#### Шаг 4. Объединение данных

In [ ]:
#Объединим данные о пользователях, поездках и подписках в один датафрейм. 
data = users.merge(rides, on='user_id')
data = data.merge(subscriptions, on='subscription_type')
data.head(10)

In [ ]:
#Сравним количество строк до объединения и после 
entries = len(rides) - len(data)
entries

Количество строк полученной таблицы совпадает с количеством строк таблиц до объединения. После объединения данных мы ничего не потеряли. 

In [ ]:
#Создадим датафрейм с данными о пользователях без подписки.
data_without_subscription = data.query("subscription_type.isin(['free'])")
data_without_subscription.head(10)

In [ ]:
#Создадим датафрейм с данными о пользователях с подпиской.
data_with_subscription = data.query("subscription_type.isin(['ultra'])")
data_with_subscription.head(10)

In [ ]:
#Построим график с двумя гистограмамми по длительности поездки у пользователей с подпиской и без. 
plt.hist(data_without_subscription['duration'], bins=100 , alpha = 0.5, label='Без подписки')
plt.hist(data_with_subscription['duration'], bins=100, alpha = 0.5, label='С подпиской')
plt.legend(loc='upper left')
plt.title("Длительности поездки у пользователей с подпиской и без")
plt.xlabel("Время за которое пользователь проехал за поездку")
plt.ylabel("Частота поездок")

plt.show()

Судя по графику пользователи с подпиской и без подписки тратят примерно одинаковое время на поездку. У людей с подпиской есть тенденция тратить на поездку от 5 минут. Люди не имеющие подписку бывают ошибочно берут самокат, поэтому у них есть небольшая доля заказов, которые длятся менее 5 минут. 

In [ ]:
#Построим график с двумя гистограмамми по продолжительности поездки в метрах у пользователей с подпиской и без. 
plt.hist(data_without_subscription['distance'], bins=100 , alpha = 0.5, label='Без подписки')
plt.hist(data_with_subscription['distance'], bins=100, alpha = 0.5, label='С подпиской')
plt.legend(loc='upper left')
plt.title("Продолжительность поездки в метрах у пользователей с подпиской и без")
plt.xlabel("Расстояние, которое пользователь проехал за поездку")
plt.ylabel("Частота поездок")

plt.show()

Как мы видим по графику в основном люди с подпиской проезжают за поездку от 2 до 5 км. У людей без подписки разброс данных возрастает от 1 до 6 км. 

Насколько видно из графиков подписка не увеличивает среднее время и растояние поездки. 

#### Шаг 5. Подсчёт выручки

In [ ]:
#Округлим продолжительность и длинну каждой поездки.
data['duration'] = np.ceil(data['duration'])
data['distance'] = np.ceil(data['distance'])

In [ ]:
#Посчитаем суммарное расстояние, количество поездок и суммарное время для каждого пользователя за каждый месяц.
total_data = pd.pivot_table(data,
                           index=['month', 'user_id'],
                           values=['distance', 'duration', 'name', 'subscription_type', 'minute_price', 'start_ride_price', 'subscription_fee'],
                           aggfunc = {'distance': sum,
                                     'duration': sum,
                                     'name': len,
                                     'subscription_type': min,
                                     'minute_price': min,
                                     'start_ride_price': min,
                                     'subscription_fee': min,})
total_data.rename(columns = {'name' : 'total_rides'}, inplace = True)
total_data.sort_values(by='month')

Теперь мы можем посмотреть какое количество поездок совершил пользователь в разбивке по месяцам. 

In [ ]:
#Заменим данные в столбце duration на целочисленные.
total_data['duration'] = total_data['duration'].astype('int')
#Добавим столбец с помесячной выручкой, которую принёс каждый пользователь
total_data['total_price'] = (total_data['total_rides'] * total_data['start_ride_price']) + (total_data['duration'] * total_data['minute_price']) + total_data['subscription_fee']

total_data

In [ ]:
total_data['total_price'].describe()

В среднем пользователи тратят около 340 рублей в месяц. Наибольшая сумма потраченная в сервисе аренды за месяц составляет 1444 рубля. 

#### Шаг 6. Проверка гипотез

##### Гипотеза №1
Тратят ли пользователи с подпиской больше времени на поездки?

Нулевая гипотеза: Средняя длительность поездки пользователей с подпиской не отличается от средней длительности поездки пользователей без подписки.

Альтернативная гипотеза: Средняя длительность поездки пользователей с подпиской больше средней длительности поездки пользователей без подписки.

In [ ]:
duration_ultra_mean = data_with_subscription['duration'].mean()
duration_free_mean = data_without_subscription['duration'].mean()
print('Среднее время у Пользователей с подпиской: ', duration_ultra_mean)
print('Среднее время у Пользователей без подписки: ', duration_free_mean)
    
alpha = 0.05

result = st.ttest_ind(data_without_subscription['duration'], data_with_subscription['duration'], alternative='less')

print('p-значение =', result.pvalue)
if result.pvalue < alpha:
    print ('Отвергаем нулевую гипотеву')
else:
    print ('Не получилось отвергнуть нулевую гипотезу')

Подписка увеличивает среднее время на поездку. Кроме того, это подтверждает и средние значения продолжительности поездок для каждой категории пользователей. 

##### Гипотеза №2
Расстояние одной поездки в 3130 метров — оптимальное с точки зрения износа самоката. Можно ли сказать, что расстояние, которое проезжают пользователи с подпиской за одну поездку, не превышает 3130 метров. 

Гипотеза 0: Среднее растояние, которое проезжают пользователи с подпиской равно 3130 метров.

Гипотеза 1: Среднее растояние пользователей с подпиской больше 3130 метров.

In [ ]:
alpha = 0.05

good_distance = 3130
result = st.ttest_1samp(data_with_subscription['distance'], good_distance, alternative='greater')

print('p-значение =', result.pvalue)
if result.pvalue < alpha:
    print ('Отвергаем нулевую гипотеву')
else:
    print ('Не получилось отвергнуть нулевую гипотезу')

Среднее растояние пользователей с подпиской не превышает 3130 метров.

##### Гипотеза №3
Помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки.

Гипотеза 0: Средняя помесячная выручка от пользователей с подпиской не отличается от средней помесячной выручки пользователей без подписки.

Гипотеза 1: Средняя помесячная выручка от пользователей с подпиской выше, чем средняя помесячная выручка пользователей без подписки. 

In [ ]:
#Разделим таблицу total_data на пользователей с подпиской и без.
data_with_subscription_full = total_data[total_data['subscription_type'] == 'ultra']
data_without_subscription_full = total_data[total_data['subscription_type'] == 'free']

#Посчитаем среднне значения выручки для пользователей с подпиской и без.
print('Средняя сумма пользователей с подпиской:', data_with_subscription_full['total_price'].mean().round())
print('Средняя сумма пользователей без подписки:', data_without_subscription_full ['total_price'].mean().round())

alpha = 0.05

result = st.ttest_ind(data_with_subscription_full['total_price'], data_without_subscription_full['total_price'], alternative='greater')

print('p-значение =', result.pvalue)
if result.pvalue < alpha:
    print ('Отвергаем нулевую гипотеву')
else:
    print ('Не получилось отвергнуть нулевую гипотезу')

Помесячная выручка от пользователей с подпиской выше, чем помесячная выручка от пользователей без подписки.

##### Гипотеза № 4
Представьте такую ситуацию: техническая команда сервиса обновила сервера, с которыми взаимодействует мобильное приложение. Она надеется, что из-за этого количество обращений в техподдержку значимо снизилось. Некоторый файл содержит для каждого пользователя данные о количестве обращений до обновления и после него. Какой тест вам понадобился бы для проверки этой гипотезы?

Для проверки данной гипотизы мы используем t-тест. 
Для этого нам нужно применить метод scipy.stats.ttest_rel()

Гипотеза 0: Количество обращений в техподдержку после сервисного обслуживания не изменилось. 

Гипотеза 1: Количество обращений в техподдержку уменьшилось после сервисного обслуживания, чем до. 

Выборки являются зависимыми, поэтому применим метод scipy.stats.ttest_rel().

#### Шаг 7. Распределения

#### Вывод

Во время работы над проектом, мы проанализировали и предобработали все имеющиеся у нас данные. Удалили явные дубликаты в файле users.

В ходе анализа данных, мы пришли к выводу, что самые поплуярные города для аренды самокатов - это Пятигорск, Екатеренбург и Ростов-на-Дону. Чуть меньше половины всех пользователей используют подписку на сервис. Средний возраст пользователя - 25 лет. Растояние и время средней поездки составляет 3 км и 17,8 минут. 

Далее мы соединили все файлы с данными и построили графики для того чтобы сравнить вреям и растояние которое люди тратят на поездку с подпиской и без. Судя по графикам, мы пришли к выводу, что подписка не увеличивает среднее время и растояние поездки.
Также мы расчитали, что в среднем пользователи тратят около 340 рублей в месяц. Наибольшая сумма потраченная в сервисе аренды за месяц составляет 1444 рубля.

В ходе работы также были проверены 3 гипотезы, такие выводы мы получили:

1. Подписка увеличивает среднее время на поездку.

2. Среднее растояние пользователей с подпиской не превышает 3130 метров.

3. Помесячная выручка от пользователей с подпиской выше, чем помесячная выручка от пользователей без подписки.
